49. 名詞間の係り受けパスの抽出

文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi
とj
（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
文節i
とj
に含まれる名詞句はそれぞれ，XとYに置換する
また，係り受けパスの形状は，以下の2通りが考えられる．

文節i
から構文木の根に至る経路上に文節j
が存在する場合: 文節i
から文節j
のパスを表示
上記以外で，文節i
と文節j
から構文木の根に至る経路上で共通の文節k
で交わる場合: 文節i
から文節k
に至る直前のパスと文節j
から文節k
に至る直前までのパス，文節k
の内容を” | “で連結して表示
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．

Xは | Yに -> 関する -> 会議で | 作り出した。
Xは | Yで | 作り出した。
Xは | Yと -> いう -> 用語を | 作り出した。
Xは | Yを | 作り出した。
Xに -> 関する -> Yで
Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。
Xに -> 関する -> 会議で | Yを | 作り出した。
Xで | Yと -> いう -> 用語を | 作り出した。
Xで | Yを | 作り出した。
Xと -> いう -> Yを

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

    def __str__(self):
        return f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}"

class Chunk:
    def __init__(self, morphs=None, dst=-1):
        self.morphs = morphs if morphs else []  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst  # 係り先文節インデックス番号
        self.srcs = []  # 係り元文節インデックス番号のリスト

    def __str__(self):
        morphs_surface = ''.join([morph.surface for morph in self.morphs])
        return f"Chunk: {morphs_surface}, dst: {self.dst}, srcs: {self.srcs}"

    def get_text(self):
        return ''.join([morph.surface for morph in self.morphs if morph.pos != '記号'])

def parse_cabocha_output(parsed_file):
    sentences = []
    with open(parsed_file, 'r', encoding='utf-8') as f:
        chunks = {}
        chunk = None
        for line in f:
            if line == 'EOS\n':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                if chunks:
                    sorted_chunks = sorted(chunks.items())
                    sentence = [chunk for idx, chunk in sorted_chunks]
                    for idx, chunk in sorted_chunks:
                        if chunk.dst != -1:
                            sentence[chunk.dst].srcs.append(idx)
                    sentences.append(sentence)
                chunks = {}
                chunk = None
            elif line[0] == '*':
                if chunk is not None:
                    chunks[chunk_idx] = chunk
                cols = line.split(' ')
                chunk_idx = int(cols[1])
                dst = int(cols[2].rstrip('D'))
                chunk = Chunk(dst=dst)
            else:
                surface, feature = line.split('\t')
                features = feature.split(',')
                if len(features) >= 7:
                    morph = Morph(surface, features[6], features[0], features[1])
                    chunk.morphs.append(morph)
    return sentences

def extract_shortest_paths(parsed_file, output_file):
    sentences = parse_cabocha_output(parsed_file)
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            for i, chunk_i in enumerate(sentence):
                if any(morph.pos == '名詞' for morph in chunk_i.morphs):
                    for j in range(i + 1, len(sentence)):
                        chunk_j = sentence[j]
                        if any(morph.pos == '名詞' for morph in chunk_j.morphs):
                            path_i = []
                            path_j = []
                            common_chunk = None

                            current_chunk = chunk_i
                            while current_chunk is not None:
                                path_i.append(current_chunk)
                                if current_chunk.dst == -1:
                                    break
                                current_chunk = sentence[current_chunk.dst]

                            current_chunk = chunk_j
                            while current_chunk is not None:
                                path_j.append(current_chunk)
                                if current_chunk.dst == -1:
                                    break
                                current_chunk = sentence[current_chunk.dst]

                            for chunk in path_i:
                                if chunk in path_j:
                                    common_chunk = chunk
                                    break

                            if common_chunk:
                                path_i = path_i[:path_i.index(common_chunk) + 1]
                                path_j = path_j[:path_j.index(common_chunk)]
                                path_j.reverse()

                                path_i_text = ' -> '.join(['X' if chunk == chunk_i else chunk.get_text() for chunk in path_i])
                                path_j_text = ' -> '.join(['Y' if chunk == chunk_j else chunk.get_text() for chunk in path_j])

                                f.write(f"{path_i_text} | {common_chunk.get_text()} | {path_j_text}\n")
                            else:
                                path_i_text = ' -> '.join(['X' if chunk == chunk_i else chunk.get_text() for chunk in path_i])
                                path_j_text = ' -> '.join(['Y' if chunk == chunk_j else chunk.get_text() for chunk in path_j])

                                f.write(f"{path_i_text} -> {path_j_text}\n")

# 指定解析結果ファイルのパス
parsed_file = 'ai.ja.txt.parsed'
# 出力ファイルのパス
output_file = 'shortest_paths.txt'

# 名詞句ペアを結ぶ最短係り受けパスを抽出してファイルに保存
extract_shortest_paths(parsed_file, output_file)
